# User Service Integration Tests

This notebook tests the user service endpoints through the API Gateway.

**Endpoints tested:**
- GET `/api/v1/users/me` - Get current user profile
- PUT `/api/v1/users/me` - Update user profile (full replace)
- PATCH `/api/v1/users/me` - Partial update user profile
- POST `/api/v1/users/pets` - Create new pet profile
- GET `/api/v1/users/pets` - List user's pets
- GET `/api/v1/users/pets/{id}` - Get specific pet
- PUT/PATCH `/api/v1/users/pets/{id}` - Update pet
- DELETE `/api/v1/users/pets/{id}` - Delete pet

**Prerequisites:** Must be authenticated (run auth service tests first to login).

## Setup and Configuration

In [1]:
import requests
import json
from datetime import datetime, date
from rich import print as rprint
from rich.console import Console
from rich.table import Table
from rich.panel import Panel
from rich.json import JSON
import time

console = Console()

# API Gateway URL
BASE_URL = "http://localhost:8001"
USER_BASE = f"{BASE_URL}/api/v1/users"
PET_BASE = f"{BASE_URL}/api/v1/pets"

TEST_USER_EMAIL = "test_user@example.com"
TEST_USER_PASSWORD = "securepassword123"

# Create session (will need JWT cookies from auth)
session = requests.Session()

# Test results tracker
test_results = {}

console.print("[green]✓[/green] Setup complete", style="bold")
console.print(f"[blue]API Gateway:[/blue] {BASE_URL}")
console.print(f"[blue]User Endpoints:[/blue] {USER_BASE}")
console.print(f"[blue]Pet Endpoints:[/blue] {PET_BASE}")

✓ Setup complete

API Gateway: http://localhost:8001

User Endpoints: http://localhost:8001/api/v1/users

Pet Endpoints: http://localhost:8001/api/v1/pets

## Helper Functions

In [2]:
def print_response(response, title="Response"):
    """Pretty print HTTP response."""
    status_color = "green" if response.status_code < 300 else "yellow" if response.status_code < 400 else "red"
    
    table = Table(title=title, show_header=True, header_style="bold magenta")
    table.add_column("Property", style="cyan", width=20)
    table.add_column("Value", style="white")
    
    table.add_row("Status Code", f"[{status_color}]{response.status_code}[/{status_color}]")
    table.add_row("URL", response.url)
    table.add_row("Time", f"{response.elapsed.total_seconds():.3f}s")
    
    console.print(table)
    
    try:
        json_data = response.json()
        console.print("\n[bold]Response Body:[/bold]")
        console.print(JSON(json.dumps(json_data, indent=2)))
    except:
        console.print(f"\n[yellow]Raw Response:[/yellow] {response.text}")
    
    console.print("\n" + "="*80 + "\n")
    return response


def test_endpoint(method, url, data=None, expect_success=True, test_name=None, **kwargs):
    """Test an endpoint and print results."""
    try:
        response = session.request(method, url, json=data, **kwargs)
        print_response(response, f"{method.upper()} {url}")
        
        passed = False
        if expect_success and response.status_code >= 400:
            console.print("[red]✗ Expected success but got error[/red]", style="bold")
        elif not expect_success and response.status_code < 400:
            console.print("[yellow]⚠ Expected error but got success[/yellow]", style="bold")
        else:
            console.print("[green]✓ Response matches expectation[/green]", style="bold")
            passed = True
        
        # Record test result if test_name provided
        if test_name:
            test_results[test_name] = {
                'passed': passed,
                'status_code': response.status_code,
                'expected_success': expect_success
            }
        
        return response
    except requests.exceptions.RequestException as e:
        console.print(f"[red]✗ Request failed: {e}[/red]", style="bold")
        if test_name:
            test_results[test_name] = {'passed': False, 'status_code': 'ERROR', 'expected_success': expect_success}
        return None


console.print("[green]✓[/green] Helper functions loaded", style="bold")

✓ Helper functions loaded

## 1. Authentication

Login to get JWT cookies for authenticated requests.

In [3]:
console.print("\n[bold cyan]Registering Test User...[/bold cyan]\n")

register_data = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
}

response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/register", data=register_data, expect_success=True, test_name="User Registration")

console.print("\n[bold cyan]Authenticating...[/bold cyan]\n")

# Use existing test user or create new one
login_data = {
    "email": TEST_USER_EMAIL,
    "password": TEST_USER_PASSWORD
}

response = test_endpoint("POST", f"{BASE_URL}/api/v1/auth/login", data=login_data, test_name="Authentication")

if response and response.status_code == 200:
    console.print("[green]✓ Authentication successful[/green]\n", style="bold")
    authenticated = True
else:
    console.print("[red]✗ Authentication failed - run auth service tests first[/red]\n", style="bold")
    authenticated = False

Registering Test User...

           POST http://localhost:8001/api/v1/auth/register           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 201                                        │
│ URL                  │ http://localhost:8001/api/v1/auth/register │
│ Time                 │ 0.392s                                     │
└──────────────────────┴────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
      "email": "test_user@example.com",
      "first_name": "",
      "last_name": "",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:25.843238Z"
}

================================================================================

...

Response Body:

{
  "success": true,
  "data": {
    "user": {
      "id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
      "email": "test_user@example.com",
      "first_name": "",
      "last_name": "",
      "role": "user",
      "is_verified": true
    }
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:26.235893Z"
}

================================================================================

✓ Response matches expectation

✓ Authentication successful

## 2. Get User Profile

Retrieve the current user's profile information.

In [4]:
console.print("\n[bold cyan]Testing Get User Profile...[/bold cyan]\n")

response = test_endpoint("GET", f"{USER_BASE}/me", test_name="Get User Profile")

if response and response.status_code == 200:
    console.print("[green]✓ Profile retrieved successfully[/green]\n", style="bold")
    current_profile = response.json().get("data", {})
else:
    console.print("[red]✗ Failed to retrieve profile[/red]\n", style="bold")
    current_profile = None

Testing Get User Profile...

           GET http://localhost:8001/api/v1/users/me            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                   │
│ URL                  │ http://localhost:8001/api/v1/users/me │
│ Time                 │ 0.030s                                │
└──────────────────────┴───────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "575aa447-a649-4b42-941b-847196131b7f",
    "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
    "phone": "",
    "address": null,
    "preferences": {},
    "created_at": "2026-02-06T13:40:26.676545Z",
    "updated_at": "2026-02-06T13:40:26.676572Z"
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:26.681057Z"
}

================================================================================

✓ Response matches expectation

✓ Profile retrieved successfully

## 3. Update User Profile (PATCH)

Partially update user profile fields.

In [5]:
console.print("\n[bold cyan]Testing Partial Profile Update (PATCH)...[/bold cyan]\n")

update_data = {
    "phone": "+1-555-0123",
    "address": "123 Test Street, Test City, TC 12345"
}

response = test_endpoint("PATCH", f"{USER_BASE}/me", data=update_data, test_name="Update Profile (PATCH)")

if response and response.status_code == 200:
    console.print("[green]✓ Profile updated successfully[/green]\n", style="bold")
    updated_profile = response.json().get("data", {})
else:
    console.print("[red]✗ Profile update failed[/red]\n", style="bold")

Testing Partial Profile Update (PATCH)...

          PATCH http://localhost:8001/api/v1/users/me           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                 ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                   │
│ URL                  │ http://localhost:8001/api/v1/users/me │
│ Time                 │ 0.035s                                │
└──────────────────────┴───────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "575aa447-a649-4b42-941b-847196131b7f",
    "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
    "phone": "+1-555-0123",
    "address": "123 Test Street, Test City, TC 12345",
    "preferences": {},
    "created_at": "2026-02-06T13:40:26.676545Z",
    "updated_at": "2026-02-06T13:40:26.742136Z"
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:26.746800Z"
}

================================================================================

✓ Response matches expectation

✓ Profile updated successfully

## 4. Create Pet Profile

Add a new pet to the user's account.

In [6]:
console.print("\n[bold cyan]Testing Create Pet Profile...[/bold cyan]\n")

pet_data = {
    "name": "Max",
    "species": "dog",
    "breed": "Golden Retriever",
    "age": 3,
    "weight": 30.5,
    "health_conditions": "Hip dysplasia - under monitoring"
}

response = test_endpoint("POST", f"{PET_BASE}", data=pet_data, test_name="Create Pet")

if response and response.status_code in [200, 201]:
    console.print("[green]✓ Pet created successfully[/green]\n", style="bold")
    created_pet = response.json().get("data", {})
    pet_id = created_pet.get("id")
else:
    console.print("[red]✗ Pet creation failed[/red]\n", style="bold")
    pet_id = None

Testing Create Pet Profile...

           POST http://localhost:8001/api/v1/pets           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 201                               │
│ URL                  │ http://localhost:8001/api/v1/pets │
│ Time                 │ 0.031s                            │
└──────────────────────┴───────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "94bbde95-cb9d-431e-94f1-1583e5ec7c00",
    "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
    "name": "Max",
    "breed": "Golden Retriever",
    "breed_confidence": null,
    "species": "dog",
    "age": 3,
    "weight": 30.5,
    "health_conditions": "Hip dysplasia - under monitoring",
    "image_url": null,
    "created_at": "2026-02-06T13:40:26.802394Z",
    "updated_at": "2026-02-06T13:40:26.802425Z"
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:26.808711Z"
}

================================================================================

✓ Response matches expectation

✓ Pet created successfully

## 5. List All Pets

Retrieve all pets for the current user.

In [7]:
console.print("\n[bold cyan]Testing List All Pets...[/bold cyan]\n")

response = test_endpoint("GET", f"{PET_BASE}", test_name="List Pets")

if response and response.status_code == 200:
    pets_data = response.json().get("data", [])
    console.print(f"[green]✓ Found {len(pets_data)} pet(s)[/green]\n", style="bold")
    
    # Display pets in table
    if pets_data:
        pets_table = Table(title="User's Pets", show_header=True, header_style="bold magenta")
        pets_table.add_column("ID", style="cyan")
        pets_table.add_column("Name", style="green")
        pets_table.add_column("Species", style="yellow")
        pets_table.add_column("Breed", style="blue")
        pets_table.add_column("Age", style="white")
        
        for pet in pets_data:
            pets_table.add_row(
                str(pet.get("id", "")),
                pet.get("name", ""),
                pet.get("species", ""),
                pet.get("breed", ""),
                str(pet.get("age", ""))
            )
        
        console.print(pets_table)
        console.print()
else:
    console.print("[red]✗ Failed to list pets[/red]\n", style="bold")

Testing List All Pets...

           GET http://localhost:8001/api/v1/pets            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                               │
│ URL                  │ http://localhost:8001/api/v1/pets │
│ Time                 │ 0.036s                            │
└──────────────────────┴───────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": [
    {
      "id": "94bbde95-cb9d-431e-94f1-1583e5ec7c00",
      "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
      "name": "Max",
      "breed": "Golden Retriever",
      "breed_confidence": null,
      "species": "dog",
      "age": 3,
      "weight": 30.5,
      "health_conditions": "Hip dysplasia - under monitoring",
      "image_url": null,
      "created_at": "2026-02-06T13:40:26.802394Z",
      "updated_at": "2026-02-06T13:40:26.802425Z"
    }
  ],
  "error": null,
  "timestamp": "2026-02-06T13:40:26.877396Z"
}

================================================================================

✓ Response matches expectation

✓ Found 1 pet(s)

                                   User's Pets                                    
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━┓
┃ ID                                   ┃ Name ┃ Species ┃ Breed            ┃ Age ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━┩
│ 94bbde95-cb9d-431e-94f1-1583e5ec7c00 │ Max  │ dog     │ Golden Retriever │ 3   │
└──────────────────────────────────────┴──────┴─────────┴──────────────────┴─────┘

## 6. Get Specific Pet

Retrieve details for a specific pet.

In [8]:
if pet_id:
    console.print(f"\n[bold cyan]Testing Get Pet by ID ({pet_id})...[/bold cyan]\n")
    
    response = test_endpoint("GET", f"{PET_BASE}/{pet_id}", test_name="Get Specific Pet")
    
    if response and response.status_code == 200:
        console.print("[green]✓ Pet details retrieved[/green]\n", style="bold")
    else:
        console.print("[red]✗ Failed to retrieve pet[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipping - no pet ID available[/yellow]\n")

Testing Get Pet by ID (94bbde95-cb9d-431e-94f1-1583e5ec7c00)...

           GET http://localhost:8001/api/v1/pets/94bbde95-cb9d-431e-94f1-1583e5ec7c00            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                                                    │
│ URL                  │ http://localhost:8001/api/v1/pets/94bbde95-cb9d-431e-94f1-1583e5ec7c00 │
│ Time                 │ 0.037s                                                                 │
└──────────────────────┴────────────────────────────────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "94bbde95-cb9d-431e-94f1-1583e5ec7c00",
    "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
    "name": "Max",
    "breed": "Golden Retriever",
    "breed_confidence": null,
    "species": "dog",
    "age": 3,
    "weight": 30.5,
    "health_conditions": "Hip dysplasia - under monitoring",
    "image_url": null,
    "created_at": "2026-02-06T13:40:26.802394Z",
    "updated_at": "2026-02-06T13:40:26.802425Z"
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:26.958454Z"
}

================================================================================

✓ Response matches expectation

✓ Pet details retrieved

## 7. Update Pet (PATCH)

Partially update pet information.

In [9]:
if pet_id:
    console.print(f"\n[bold cyan]Testing Update Pet ({pet_id})...[/bold cyan]\n")
    
    pet_update = {
        "age": 4,
        "weight": 32.0,
        "health_conditions": "Hip dysplasia - improved with supplements"
    }
    
    response = test_endpoint("PATCH", f"{PET_BASE}/{pet_id}", data=pet_update, test_name="Update Pet (PATCH)")
    
    if response and response.status_code == 200:
        console.print("[green]✓ Pet updated successfully[/green]\n", style="bold")
    else:
        console.print("[red]✗ Pet update failed[/red]\n", style="bold")
else:
    console.print("[yellow]⚠ Skipping - no pet ID available[/yellow]\n")

Testing Update Pet (94bbde95-cb9d-431e-94f1-1583e5ec7c00)...

          PATCH http://localhost:8001/api/v1/pets/94bbde95-cb9d-431e-94f1-1583e5ec7c00           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                                                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 200                                                                    │
│ URL                  │ http://localhost:8001/api/v1/pets/94bbde95-cb9d-431e-94f1-1583e5ec7c00 │
│ Time                 │ 0.041s                                                                 │
└──────────────────────┴────────────────────────────────────────────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "94bbde95-cb9d-431e-94f1-1583e5ec7c00",
    "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
    "name": "Max",
    "breed": "Golden Retriever",
    "breed_confidence": null,
    "species": "dog",
    "age": 4,
    "weight": 32.0,
    "health_conditions": "Hip dysplasia - improved with supplements",
    "image_url": null,
    "created_at": "2026-02-06T13:40:26.802394Z",
    "updated_at": "2026-02-06T13:40:27.030944Z"
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:27.035123Z"
}

================================================================================

✓ Response matches expectation

✓ Pet updated successfully

## 8. Create Multiple Pets

Test creating multiple pets with different species.

In [10]:
console.print("\n[bold cyan]Testing Multiple Pet Creation...[/bold cyan]\n")

test_pets = [
    {
        "name": "Whiskers",
        "species": "cat",
        "breed": "Persian",
        "age": 2,
        "weight": 4.5
    },
    {
        "name": "Buddy",
        "species": "dog",
        "breed": "Labrador",
        "age": 5,
        "weight": 35.0
    },
    {
        "name": "Tweety",
        "species": "other",
        "breed": "Parakeet (bird)",
        "age": 1,
        "weight": 0.03
    }
]

created_pet_ids = []
multiple_pets_success = True

for pet in test_pets:
    console.print(f"[blue]Creating pet: {pet['name']} ({pet['species']})[/blue]")
    response = session.post(f"{PET_BASE}", json=pet)
    
    if response.status_code in [200, 201]:
        pet_data = response.json().get("data", {})
        created_pet_ids.append(pet_data.get("id"))
        console.print(f"  [green]✓ Created with ID: {pet_data.get('id')}[/green]")
    else:
        console.print(f"  [red]✗ Failed to create {pet['name']}[/red]")
        multiple_pets_success = False

# Record test result
test_results['Multiple Pets'] = {
    'passed': multiple_pets_success,
    'status_code': 'OK' if multiple_pets_success else 'FAILED',
    'expected_success': True
}

console.print(f"\n[green]✓ Created {len(created_pet_ids)} additional pets[/green]\n", style="bold")

Testing Multiple Pet Creation...

Creating pet: Whiskers (cat)

✓ Created with ID: 81be9593-6afd-42e3-acc3-596971f7cf0a

Creating pet: Buddy (dog)

✓ Created with ID: 2e70be8f-1bed-43bc-9bc4-58f66a3a3c9a

Creating pet: Tweety (other)

✓ Created with ID: 7798f92c-28a2-4a1c-b4f1-4c35ab38c766

✓ Created 3 additional pets

## 9. Test Authorization

Verify that users cannot access other users' pets.

In [11]:
console.print("\n[bold cyan]Testing Authorization (Access Other User's Pet)...[/bold cyan]\n")

# Try to access a pet with a non-existent ID (simulates other user's pet)
fake_pet_id = 99999

response = test_endpoint("GET", f"{PET_BASE}/{fake_pet_id}", expect_success=False, test_name="Authorization")

auth_test_passed = response.status_code in [404, 403]

if auth_test_passed:
    console.print("[green]✓ Correctly denied access to non-owned pet[/green]\n", style="bold")
else:
    console.print("[yellow]⚠ Authorization check did not work as expected[/yellow]\n", style="bold")

Testing Authorization (Access Other User's Pet)...

           GET http://localhost:8001/api/v1/pets/99999            
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 404                                     │
│ URL                  │ http://localhost:8001/api/v1/pets/99999 │
│ Time                 │ 0.054s                                  │
└──────────────────────┴─────────────────────────────────────────┘

Response Body:

{
  "success": false,
  "data": null,
  "error": {
    "code": "NOT_FOUND",
    "message": "The requested resource was not found",
    "details": {}
  },
  "timestamp": "2026-02-06T13:40:27.337300"
}

================================================================================

✓ Response matches expectation

✓ Correctly denied access to non-owned pet

## 10. Test Input Validation

Verify that invalid data is rejected.

In [12]:
console.print("\n[bold cyan]Testing Input Validation...[/bold cyan]\n")

# Invalid pet data (missing required fields)
invalid_pet = {
    "name": "Invalid",
    # Missing species and breed
}

console.print("[blue]Sending invalid pet data (missing required fields)...[/blue]\n")
response = test_endpoint("POST", f"{PET_BASE}", data=invalid_pet, expect_success=False, test_name="Input Validation")

validation_test_passed = response.status_code == 422

if validation_test_passed:
    console.print("[green]✓ Correctly rejected invalid data[/green]\n", style="bold")
else:
    console.print("[yellow]⚠ Validation did not work as expected[/yellow]\n", style="bold")

Testing Input Validation...

Sending invalid pet data (missing required fields)...

           POST http://localhost:8001/api/v1/pets           
┏━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property             ┃ Value                             ┃
┡━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Status Code          │ 201                               │
│ URL                  │ http://localhost:8001/api/v1/pets │
│ Time                 │ 0.077s                            │
└──────────────────────┴───────────────────────────────────┘

Response Body:

{
  "success": true,
  "data": {
    "id": "f475432e-5417-4c5e-b7a0-8dc851129cd3",
    "user_id": "90c52057-bd7a-4e39-9cb0-8c1b939ead47",
    "name": "Invalid",
    "breed": "",
    "breed_confidence": null,
    "species": "dog",
    "age": null,
    "weight": null,
    "health_conditions": [],
    "image_url": null,
    "created_at": "2026-02-06T13:40:27.382607Z",
    "updated_at": "2026-02-06T13:40:27.382638Z"
  },
  "error": null,
  "timestamp": "2026-02-06T13:40:27.388367Z"
}

================================================================================

⚠ Expected error but got success

⚠ Validation did not work as expected

## 11. Cleanup - Delete Test Pets and User

Delete all pets created during testing.

In [13]:
console.print("\n[bold cyan]Cleaning Up Test Pets...[/bold cyan]\n")

# Collect all pet IDs
all_pet_ids = [pet_id] + created_pet_ids if pet_id else created_pet_ids

deleted_count = 0
cleanup_success = True

for pid in all_pet_ids:
    if pid:
        response = session.delete(f"{PET_BASE}/{pid}")
        if response.status_code in [200, 204]:
            console.print(f"[green]✓ Deleted pet ID: {pid}[/green]")
            deleted_count += 1
        else:
            console.print(f"[red]✗ Failed to delete pet ID: {pid}[/red]")
            cleanup_success = False

# Record test result
test_results['Cleanup'] = {
    'passed': cleanup_success,
    'status_code': 'OK' if cleanup_success else 'FAILED',
    'expected_success': True
}

console.print(f"\n[blue]Deleted {deleted_count} test pets[/blue]\n", style="bold")

# Delete test user
console.print("\n[bold cyan]Deleting test user...[/bold cyan]\n")
AUTH_BASE = f"{BASE_URL}/api/v1/auth"
delete_response = session.delete(f"{AUTH_BASE}/delete")

if delete_response.status_code == 200:
    data = delete_response.json()
    console.print(f"[green]✓ {data.get('data', {}).get('message', 'User deleted')}[/green]\n", style="bold")
else:
    console.print(f"[yellow]⚠ Failed to delete test user: {delete_response.status_code}[/yellow]\n", style="bold")


Cleaning Up Test Pets...

✓ Deleted pet ID: 94bbde95-cb9d-431e-94f1-1583e5ec7c00

✓ Deleted pet ID: 81be9593-6afd-42e3-acc3-596971f7cf0a

✓ Deleted pet ID: 2e70be8f-1bed-43bc-9bc4-58f66a3a3c9a

✓ Deleted pet ID: 7798f92c-28a2-4a1c-b4f1-4c35ab38c766

Deleted 4 test pets

Deleting test user...

✓ User account test_user@example.com deleted successfully

## 12. Test Summary

In [14]:
console.print("\n" + "="*80, style="bold")
console.print("[bold cyan]User Service Test Summary[/bold cyan]")
console.print("="*80 + "\n", style="bold")

summary_table = Table(show_header=True, header_style="bold magenta")
summary_table.add_column("Test", style="cyan", width=40)
summary_table.add_column("Status", style="white", width=20)
summary_table.add_column("Notes", style="white")

# Define expected test keys and their display info
test_info = {
    'Authentication': 'JWT cookies valid',
    'Get User Profile': 'Profile retrieved',
    'Update Profile (PATCH)': 'Partial update successful',
    'Create Pet': 'Pet profile created',
    'List Pets': 'All pets retrieved',
    'Get Specific Pet': 'Pet details retrieved',
    'Update Pet (PATCH)': 'Pet info updated',
    'Multiple Pets': f'Created {len(created_pet_ids)} pets',
    'Authorization': 'Access control working',
    'Input Validation': 'Invalid data rejected',
    'Cleanup': f'Deleted {deleted_count} pets'
    'Cascade User Deletion': 'Full cascade deletion test',
}

# Build summary from actual test results
passed_count = 0
failed_count = 0

for test_name, notes in test_info.items():
    if test_name in test_results:
        result = test_results[test_name]
        if result['passed']:
            summary_table.add_row(test_name, "[green]✓ Pass[/green]", notes)
            passed_count += 1
        else:
            status_msg = f"✗ Fail (status: {result.get('status_code', 'N/A')})"
            summary_table.add_row(test_name, f"[red]{status_msg}[/red]", notes)
            failed_count += 1
    else:
        summary_table.add_row(test_name, "[yellow]⚠ Not Run[/yellow]", notes)
        failed_count += 1

console.print(summary_table)

# Overall result
if failed_count == 0:
    console.print(f"\n[bold green]All {passed_count} user service tests passed![/bold green]\n")
else:
    console.print(f"\n[bold red]{failed_count} test(s) failed, {passed_count} passed[/bold red]\n")

SyntaxError: invalid syntax (3795915737.py, line 23)

## Test 12: Cascade User Deletion

Test that deleting a user account cascades across microservices:
1. Creates a new test user with profile and pets
2. Deletes the user via `/api/v1/auth/delete`
3. Verifies deletion summary includes data from both auth-service and user-service
4. Confirms user cannot login after deletion

In [ ]:
console.print("\n" + "="*80, style="bold")
console.print("[bold cyan]Test 12: Cascade User Deletion[/bold cyan]")
console.print("="*80 + "\n", style="bold")

# Step 1: Create a new test user specifically for deletion testing
console.print("[yellow]Step 1: Creating new test user for deletion...[/yellow]")
delete_test_email = f"delete_test_{int(time.time())}@example.com"
delete_test_password = "DeleteTest123"

register_response = session.post(
    f"{BASE_URL}/api/v1/auth/register",
    json={
        "email": delete_test_email,
        "password": delete_test_password,
        "first_name": "Delete",
        "last_name": "Test"
    }
)

if register_response.status_code == 201:
    user_data = register_response.json()['data']['user']
    delete_user_id = user_data['id']
    console.print(f"[green]✓[/green] Test user created: {delete_test_email}")
    console.print(f"  User ID: {delete_user_id}")
else:
    console.print(f"[red]✗ Failed to create test user: {register_response.status_code}[/red]")
    display_response(register_response)

# Step 2: Create user profile
console.print("\n[yellow]Step 2: Creating user profile...[/yellow]")
profile_response = session.put(
    f"{USER_BASE}/me",
    json={
        "bio": "This user will be deleted for testing cascade deletion",
        "phone": "+1234567890"
    }
)

if profile_response.status_code == 200:
    console.print("[green]✓[/green] User profile created")
else:
    console.print(f"[red]✗ Failed to create profile: {profile_response.status_code}[/red]")

# Step 3: Create multiple pets
console.print("\n[yellow]Step 3: Creating pets for test user...[/yellow]")
test_pets = [
    {"name": "DeleteDog1", "species": "dog", "breed": "Labrador", "age": 3},
    {"name": "DeleteCat1", "species": "cat", "breed": "Siamese", "age": 2}
]

created_test_pets = []
for pet_data in test_pets:
    pet_response = session.post(f"{USER_BASE}/me/pets", json=pet_data)
    if pet_response.status_code == 201:
        pet_id = pet_response.json()['data']['id']
        created_test_pets.append(pet_id)
        console.print(f"[green]✓[/green] Pet created: {pet_data['name']} (ID: {pet_id})")
    else:
        console.print(f"[red]✗ Failed to create pet: {pet_data['name']}[/red]")

console.print(f"\n[blue]Created {len(created_test_pets)} pets for deletion test[/blue]")

# Step 4: Delete the user account (cascade deletion)
console.print("\n[yellow]Step 4: Deleting user account (cascade)...[/yellow]")
delete_response = session.delete(f"{BASE_URL}/api/v1/auth/delete")

cascade_test_passed = False
deletion_summary = None

if delete_response.status_code == 200:
    delete_data = delete_response.json()
    if delete_data.get('success'):
        console.print(f"[green]✓[/green] User account deleted successfully")
        
        # Display deletion summary
        if 'deleted' in delete_data.get('data', {}):
            deletion_summary = delete_data['data']['deleted']
            console.print("\n[bold cyan]Deletion Summary:[/bold cyan]")
            
            summary_table = Table(show_header=True, header_style="bold magenta")
            summary_table.add_column("Service", style="cyan")
            summary_table.add_column("Resource Type", style="yellow")
            summary_table.add_column("Count", style="green", justify="right")
            
            # User service deletions
            if 'user_service' in deletion_summary:
                user_svc_data = deletion_summary['user_service']
                summary_table.add_row("user-service", "profiles", str(user_svc_data.get('profiles', 0)))
                summary_table.add_row("user-service", "pets", str(user_svc_data.get('pets', 0)))
                summary_table.add_row("user-service", "analyses", str(user_svc_data.get('analyses', 0)))
            
            # Auth service deletions
            if 'auth_service' in deletion_summary:
                auth_svc_data = deletion_summary['auth_service']
                summary_table.add_row("auth-service", "users", str(auth_svc_data.get('users', 0)))
                summary_table.add_row("auth-service", "refresh_tokens", str(auth_svc_data.get('refresh_tokens', 0)))
            
            console.print(summary_table)
            
            # Verify cascade deletion worked correctly
            user_svc = deletion_summary.get('user_service', {})
            auth_svc = deletion_summary.get('auth_service', {})
            
            expected_profiles = 1
            expected_pets = len(created_test_pets)
            expected_users = 1
            
            if (user_svc.get('profiles', 0) == expected_profiles and
                user_svc.get('pets', 0) == expected_pets and
                auth_svc.get('users', 0) == expected_users):
                console.print("\n[green]✓ Cascade deletion verified: All data removed from both services[/green]")
                cascade_test_passed = True
            else:
                console.print("\n[red]✗ Cascade deletion incomplete: Counts don't match expected[/red]")
                console.print(f"  Expected: {expected_profiles} profile, {expected_pets} pets, {expected_users} user")
        else:
            console.print("[yellow]⚠ Deletion successful but no summary provided[/yellow]")
            cascade_test_passed = True  # Still consider it passed if deletion succeeded
    else:
        console.print(f"[red]✗ Deletion failed: {delete_data.get('error', {}).get('message')}[/red]")
else:
    console.print(f"[red]✗ Delete request failed with status {delete_response.status_code}[/red]")
    display_response(delete_response)

# Step 5: Verify user cannot login anymore
console.print("\n[yellow]Step 5: Verifying user is deleted (login should fail)...[/yellow]")
login_response = session.post(
    f"{BASE_URL}/api/v1/auth/login",
    json={
        "email": delete_test_email,
        "password": delete_test_password
    }
)

if login_response.status_code in [401, 403]:
    console.print("[green]✓[/green] User cannot login (confirmed deleted)")
    login_verified = True
else:
    console.print(f"[red]✗ User can still login (status: {login_response.status_code})[/red]")
    login_verified = False

# Record test result
final_result = cascade_test_passed and login_verified
test_results['Cascade User Deletion'] = {
    'passed': final_result,
    'status_code': delete_response.status_code,
    'deletion_summary': deletion_summary,
    'login_verified': login_verified
}

if final_result:
    console.print("\n[bold green]✓ Cascade User Deletion Test PASSED[/bold green]")
else:
    console.print("\n[bold red]✗ Cascade User Deletion Test FAILED[/bold red]")

console.print("\n" + "="*80 + "\n", style="bold")

## Final Cleanup

In [ ]:
# Clear session
session.cookies.clear()
console.print("[blue]Session cleared[/blue]")


Session cleared